In [82]:
import pandas as pd
import numpy as np
from datetime import datetime
import utils
import importlib

importlib.reload(utils)

<module 'utils' from '/Users/devyn/Computer Science/Data Science/da8_personal_project/utils.py'>

# Personal Strava EDA  

Write introduction and author here...

Write about `data` and attributes here...

## Collecting and Cleaning the Data  

Write the ways in which you are cleaning the data...  

In [83]:
utils.collectStravaData()

utils.collectWeatherData()


/Users/devyn/Computer Science/Data Science/da8_personal_project/utils.py:136: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned["Activity Date"] = pd.to_datetime(df_cleaned["Activity Date"])


## Strava Activity Stats  

Write here why you chose these statistics...

In [84]:
stats_ser = utils.calculateStravaStats()
print(stats_ser)

Total Activities                    158.00
Average Active Time (in minutes)     63.81
Average Heart Rate                  141.95
Average Relative Effort              70.09
Relative Effort StDev                49.98
Total Rowing Activities              96.00
Average RE for Rowing                83.80
Total Ride Activities                22.00
Average RE for Riding                60.41
dtype: float64


For aggregation and hypothesis, maybe I can see if there is a correlatino between shorter workouts and higher heart rate 

## Visualization

## Merged Data  

Explain how you are merging the data here...

In [85]:
strava_df = pd.read_csv("devyn_CLEANED_strava_activities.csv")
weather_df = pd.read_csv("Spokane_CLEANED_daily_weather.csv")

merged_df = pd.merge(strava_df, weather_df, on="Activity Date", how="inner")
merged_df.to_csv("merged_data.csv")